### Importing Libraries and Configuring the Setup
Here I bring in all the libraries I’ll need — pandas, numpy, matplotlib, etc. — for data handling, visualization, and optimization.
I also set up file paths and define rules like maximum stock weight and the tilt towards Consumer and Finance sectors during festive months.
This is the backbone of the project — once this is set, the rest of the analysis flows smoothly.

In [ ]:
# Cell A - imports & config
import os, math, warnings
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize

warnings.filterwarnings("ignore")
plt.style.use("seaborn-v0_8")

# Config (tweak)
DATA_DIR = "data/prices"
OUTPUT_DIR = "output"
EXCEL_PATH = "Tickers.csv"   # path to your NSE file screenshot showed
MAX_WEIGHT_PER_STOCK = 0.10
ANNUALIZE = 252
TILT_MONTHS = [9,10,11,12]   # Sep-Dec (festive season)
TILT_SHARE = 0.12            # increase total allocation to target sectors by this absolute share (e.g., 0.12 = +12%)
TARGET_SECTORS = ["Consumer","Finance"]  # sector names to tilt toward
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

###  Loading Tickers
This part is about loading the tickers from a CSV file.  
I also add some error handling so the code doesn’t crash if the file is missing or misnamed.  
It’s a small step, but important because without a clean list of tickers, nothing else can move forward.  

In [ ]:
# Cell B: Read tickers safely

import pandas as pd

def load_tickers():
    try:
        # Always read from tickers.csv
        df = pd.read_csv("tickers.csv")
        print(f"[ok] Loaded {len(df)} tickers from tickers.csv")
        return df
    except FileNotFoundError:
        raise FileNotFoundError("tickers.csv not found! Please save the file in the same folder.")
    except Exception as e:
        raise RuntimeError(f"Unexpected error reading tickers.csv: {e}")

# Test
tickers_df = load_tickers()
display(tickers_df.head())

### Cleaning and Preparing Ticker Data
This step standardizes the ticker symbols by removing any unwanted suffixes (like `.NS`) and ensures consistency.  
The cleaned data is saved back for future steps. Having clean ticker data is critical because mismatches can lead to errors when merging or fetching stock prices.


In [ ]:
# Load your tickers file
tickers_df = pd.read_csv(EXCEL_PATH)  # or pd.read_excel() if you're using Excel

# ✅ Clean column names (remove \n, tabs, extra spaces)
tickers_df.columns = (
    tickers_df.columns
    .str.replace(r'\s+', ' ', regex=True)  # Replace \n, \t, and multiple spaces with a single space
    .str.strip()                           # Remove leading/trailing spaces
)

tickers_df['TICKER'] = tickers_df.iloc[:, 0].astype(str).str.strip() + ".NS"
tickers_df.to_csv(EXCEL_PATH, index=False)


display(tickers_df.head())


### Cell 4: Adding Sector Information
I link the tickers with their sector data and remove entries like “NIFTY 100” since that’s not a tradable stock.  
This step matters because my whole project revolves around sector tilts, so I need every stock tied to the right sector.  

In [ ]:
tickers_df = pd.read_csv("Tickers.csv")
sectors_df = pd.read_csv("sectors.csv")

# Clean tickers_df column names: remove newline and extra whitespace
tickers_df.columns = tickers_df.columns.str.replace('\n', '', regex=True).str.strip()

# Clean sectors_df columns similarly (just in case)
sectors_df.columns = sectors_df.columns.str.replace('\n', '', regex=True).str.strip()

# At this point, tickers_df.columns should include 'SYMBOL' exactly
print("After cleaning tickers:", tickers_df.columns.tolist())


# Then drop 'NIFTY 100'
tickers_df = tickers_df[tickers_df['Symbol'] != 'NIFTY 100']


In [ ]:
tickers_df['Industry'].value_counts().head(17)

###  Checking Industry Spread
Here I quickly check how many companies belong to each industry.  
It helps me see if Consumer and Finance are well represented — because if they’re not, tilting towards them wouldn’t mean much.  

In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime

# Example: Top Indian stocks (same as in your dataset)
stocks = ["RELIANCE.NS", "HDFCBANK.NS", "ICICIBANK.NS",
          "INFY.NS", "TCS.NS", "KOTAKBANK.NS",
          "HINDUNILVR.NS", "ITC.NS", "LT.NS", "SBIN.NS"]

# Fetch data from Jan 2022 till today
data = yf.download(stocks, start="2019-01-01", end=datetime.today().strftime('%Y-%m-%d'))['Close']

# Reset index to keep Date column
data = data.reset_index()
print(data.tail())   # check latest rows


###  Testing Price Downloads
Before I go all in, I test downloading price data for a few well-known Indian stocks.  
This shows me that Yahoo Finance is working as expected and I can get the historical data I’ll need.  

In [ ]:
def load_all_prices(data_dir=DATA_DIR):
    price_files = glob(os.path.join(data_dir, "*.parquet"))
    price_data = []

    for file in price_files:
        try:
            df = pd.read_parquet(file)
            print(f"Columns in {file}: {df.columns.tolist()}")  # Debug

            if df.empty:
                print(f"Skipping {file}, dataframe empty")
                continue

            if 'Close' not in df.columns:
                print(f"Skipping {file}, no 'Close' column")
                continue

            ticker = os.path.basename(file).replace(".parquet", "").replace("_", ".")
            
            temp_df = df[['Close']].copy()
            temp_df['Ticker'] = ticker
            temp_df['Date'] = temp_df.index
            temp_df.rename(columns={'Close': 'Price'}, inplace=True)

            # Check columns after rename
            print(f"Columns in temp_df before append: {temp_df.columns.tolist()}")

            price_data.append(temp_df)
        except Exception as e:
            print(f"[error] Failed to load {file}: {e}")

    if price_data:
        all_prices = pd.concat(price_data, ignore_index=True)
        print("Columns after concat:", all_prices.columns.tolist())  # Debug
        
        if 'Price' not in all_prices.columns:
            print("ERROR: 'Price' column not found in concatenated DataFrame")
            return pd.DataFrame()

        all_prices['Date'] = pd.to_datetime(all_prices['Date'])
        
        # Print a sample before dropping NA
        print("Sample before dropping NA:")
        print(all_prices.head())

        all_prices.dropna(subset=['Price'], inplace=True)

        print("Sample after dropping NA:")
        print(all_prices.head())
        
        return all_prices
    else:
        print("No valid price data found.")
        return pd.DataFrame()


###  Building a Price Loader
This function goes through all the stored parquet files, picks out the prices, and combines them into one dataset.  
Instead of dealing with scattered files, I now have everything in a single, clean DataFrame — which is much easier to work with.  

In [ ]:
import yfinance as yf

# Fetch sample data
df = yf.download("AAPL", start="2023-01-01", end="2023-01-10")

# Show the columns
print(df.columns)


###  Checking Yahoo Finance Data Format
I quickly download data for Apple (AAPL) just to see the structure of Yahoo Finance data.  
It’s a sanity check to confirm I know which columns to use, especially the “Close” price

In [ ]:

file = "data/prices/ABB_NS.parquet"
df = pd.read_parquet(file)
print(df.columns)
df.head()

###  Looking Inside a Local File
Here I open one of my stored parquet files to make sure it’s in the right format.  
This way, I don’t waste time running the full pipeline only to discover later that the files were saved incorrectly.  


In [ ]:
import os
from glob import glob
import pandas as pd

def load_all_prices(data_dir):
    price_files = glob(os.path.join(data_dir, "*.parquet"))
    price_data = []

    print(f"Looking in: {data_dir}")
    print(f"Found parquet files: {price_files}")

    for file in price_files:
        try:
            df = pd.read_parquet(file)
            print(f"Columns in {file}: {df.columns}")  # MultiIndex or not?

            if df.empty:
                print(f"Skipping {file}, dataframe empty")
                continue

            # Handle MultiIndex columns
            if isinstance(df.columns, pd.MultiIndex):
                # Get ticker name from columns (second level of MultiIndex)
                tickers_in_file = list(set([col[1] for col in df.columns if col[0] == 'Close']))
                if not tickers_in_file:
                    print(f"Skipping {file}, no 'Close' column in MultiIndex")
                    continue

                ticker = tickers_in_file[0]  # Assuming one ticker per file
                df_close = df[('Close', ticker)].copy().to_frame(name='Price')

            else:
                if 'Close' not in df.columns:
                    print(f"Skipping {file}, no 'Close' column")
                    continue
                df_close = df[['Close']].copy()
                df_close.rename(columns={'Close': 'Price'}, inplace=True)

                # Extract ticker from filename
                ticker = os.path.basename(file).replace(".parquet", "").replace("_", ".")

            df_close['Ticker'] = ticker
            df_close['Date'] = df_close.index

            print(f"Processed {file} -> Columns: {df_close.columns.tolist()}")
            price_data.append(df_close)

        except Exception as e:
            print(f"[error] Failed to load {file}: {e}")

    if price_data:
        all_prices = pd.concat(price_data, ignore_index=True)
        all_prices['Date'] = pd.to_datetime(all_prices['Date'])
        all_prices.dropna(subset=['Price'], inplace=True)

        print("Final sample:")
        print(all_prices.head())

        return all_prices
    else:
        print("No valid price data found.")
        return pd.DataFrame()


###  Improving the Price Loader
I upgrade my price loader so it can handle different file formats, including ones with MultiIndex columns.  
This makes the pipeline more robust — no matter how the data is stored, I’ll still be able to pull out the prices I need.  

In [ ]:

df_all = load_all_prices(DATA_DIR)
print(df_all.head())


###  Loading All Price Data
In this step, I loaded all the stored price data from the parquet files into a single DataFrame using the `load_all_prices()` function.  
This gave me one consolidated dataset (`df_all`) where each ticker’s historical prices were available in a consistent format.  
By printing the first few rows, I confirmed that the loading process worked correctly and that the dataset structure was as expected before moving forward.  


In [ ]:
# Make sure TICKER and Ticker columns are string
tickers_df['TICKER'] = tickers_df['TICKER'].astype(str)
df_all['Ticker'] = df_all['Ticker'].astype(str)

# Create mapping dictionaries
ticker_to_sector = tickers_df.set_index('TICKER')['Industry'].to_dict()
ticker_to_name = tickers_df.set_index('TICKER')['Company Name'].to_dict()

# Map to df_all
df_all['Sector'] = df_all['Ticker'].map(ticker_to_sector)
df_all['Company Name'] = df_all['Ticker'].map(ticker_to_name)

# Check for tickers that didn't get mapped
missing_sector = df_all[df_all['Sector'].isna()]['Ticker'].unique()
missing_name = df_all[df_all['Company Name'].isna()]['Ticker'].unique()

if len(missing_sector) > 0 or len(missing_name) > 0:
    print("Tickers missing sector or company name info:")
    print("Missing Sector:", missing_sector)
    print("Missing Company Name:", missing_name)
else:
    print("✅ All tickers successfully mapped to sector and company name.")

# Optional: check a sample
print(df_all[['Ticker', 'Sector', 'Company Name','Price']].drop_duplicates().tail())


###  Mapping Tickers to Sector and Company Names
In this step, I ensured that both the `TICKER` and `Ticker` columns were stored as strings to avoid mismatches during merging.  
I then created mapping dictionaries to connect each ticker with its corresponding sector (industry) and company name from the reference dataset.  
After applying these mappings to the main DataFrame, I checked for any tickers that failed to map correctly.  
This validation confirmed that almost all tickers were enriched with the correct sector and company information, which was essential for the later sector-based analysis and portfolio tilting.    


In [ ]:
import os
from glob import glob


# Load your master tickers file
tickers_df = pd.read_csv("tickers.csv")

# Create mapping
ticker_to_sector = tickers_df.set_index('TICKER')['Industry'].to_dict()
ticker_to_name = tickers_df.set_index('TICKER')['Company Name'].to_dict()

# Loop through each parquet file and update
for filepath in glob(os.path.join(DATA_DIR, "*.parquet")):
    try:
        df = pd.read_parquet(filepath)

        if isinstance(df.columns, pd.MultiIndex):
            # Get ticker from MultiIndex
            tickers = [col[1] for col in df.columns if col[0] == 'Close']
            if not tickers:
                continue
            ticker = tickers[0]
            df_close = df[('Close', ticker)].copy().to_frame(name='Price')
        else:
            if 'Close' not in df.columns:
                continue
            df_close = df[['Close']].copy()
            df_close.rename(columns={'Close': 'Price'}, inplace=True)
            ticker = os.path.basename(filepath).replace(".parquet", "").replace("_", ".")

        # Add sector and company info
        df_close['Ticker'] = ticker
        df_close['Sector'] = ticker_to_sector.get(ticker)
        df_close['Company Name'] = ticker_to_name.get(ticker)
        df_close['Date'] = df_close.index

        # Save updated DataFrame to same file (overwrite)
        df_close.to_parquet(filepath, index=True)
        print(f"Updated and saved: {filepath}")

    except Exception as e:
        print(f"[Error] Could not process {filepath}: {e}")


###  Updating Parquet Files with Sector and Company Info
Here I enriched each parquet price file with metadata about the stock’s sector and company name.  
I first built mapping dictionaries from the master tickers file, then looped through each parquet file to extract the closing prices and associate them with the correct ticker.  
If the file used a MultiIndex format (as Yahoo Finance sometimes provides), I handled that separately to ensure the “Close” column was correctly identified.  
Finally, I appended the sector, company name, and date information to each dataset and overwrote the parquet files with the enriched version.  
This ensured that all future analysis would directly have sector and company context available without requiring repeated joins or lookups.  


In [ ]:
def sector_performance_periods(df_all):
    import pandas as pd
    
    # Define lookback periods in days approx
    periods = {
        '3M': 90,
        '6M': 180,
        '1Y': 365,
        '3Y': 365*3
    }
    
    latest_date = df_all['Date'].max()
    results = {}

    # Make sure data is sorted
    df_all = df_all.sort_values(['Ticker', 'Date'])
    
    for label, days in periods.items():
        start_date = latest_date - pd.Timedelta(days=days)
        df_period = df_all[df_all['Date'] >= start_date]
        
        # Calculate cumulative return per ticker = (last price / first price) - 1
        first_prices = df_period.groupby('Ticker').first()['Price']
        last_prices = df_period.groupby('Ticker').last()['Price']
        cumulative_returns = (last_prices / first_prices) - 1
        
        # Map sectors for tickers
        ticker_sector = df_all.set_index('Ticker')['Sector'].drop_duplicates()
        
        # Combine returns & sectors into one DataFrame
        returns_df = pd.DataFrame({
            'Return': cumulative_returns,
            'Sector': ticker_sector
        }).dropna()
        
        # Calculate average return per sector
        sector_returns = returns_df.groupby('Sector')['Return'].mean()
        
        # Convert to percentage and round to 2 decimals
        sector_returns = (sector_returns * 100).round(2)
        
        results[label] = sector_returns.sort_values(ascending=False)
    
    # Convert results dict to DataFrame for easy viewing
    sector_perf_df = pd.DataFrame(results)
    
    return sector_perf_df

# Usage:
sector_returns = sector_performance_periods(df_all)
print(sector_returns)


###  Calculating Sector Performance Over Different Periods
In this step, I created a function to measure sector-level performance over multiple timeframes (3 months, 6 months, 1 year, and 3 years).  
For each period, I calculated the cumulative return of every stock and then averaged these returns within each sector.  
The results were organized into a single DataFrame where I could compare sectors side by side across different horizons.  
This was crucial for identifying which sectors consistently outperformed and for testing whether Consumer and Finance showed strength around festive periods, directly supporting my hypothesis.


In [ ]:
strong_sectors = sector_returns[(sector_returns['3Y'] > 30.00) & (sector_returns['3Y'] < 100.00)].sort_values(by='3Y', ascending=False)
print(strong_sectors)



### Filtering Strong Sectors Based on 3-Year Returns  

In this step, I filtered the sectors that had delivered cumulative returns between **30% and 100% over the past 3 years**.  
This helped narrow the analysis to industries that showed **consistent long-term growth** without being overly inflated.  

The following sectors qualified as strong performers (sorted by their 3-year returns in descending order):  

1. **Realty** – 94.05%  
2. **Construction** – 85.74%  
3. **Healthcare** – 78.40%  
4. **Fast Moving Consumer Goods (FMCG)** – 77.31%  
5. **Information Technology (IT)** – 71.90%  
6. **Capital Goods** – 57.30%  
7. **Services** – 50.38%  
8. **Financial Services** – 33.95%  

#### Why this was done  
The aim was to focus only on **high-performing but sustainable sectors** for further exploration. By filtering out low or excessively high growth, the analysis concentrated on industries that had proven resilience and potential for continued momentum.  

#### Insight Provided  
This filtering step proved that several critical sectors like **Realty, Construction, and Healthcare** had not only survived market fluctuations but had also significantly outperformed, thereby supporting the hypothesis that **sectoral strength drives long-term investment opportunities**.  


In [ ]:
import plotly.graph_objects as go
import pandas as pd

# Extract sector names directly from the strong_sectors DataFrame
sector_list = strong_sectors.index.tolist()

# Filter for only selected sectors
df_selected = df_all[df_all['Sector'].isin(sector_list)].copy()

# Ensure 'Date' column is datetime
df_selected['Date'] = pd.to_datetime(df_selected['Date'])

# Optional: filter by date range (auto from 2015 onward)
if not df_selected.empty:
    min_date = df_selected['Date'].min()
    cutoff_date = pd.Timestamp("2015-01-01")
    df_selected = df_selected[df_selected['Date'] >= cutoff_date]

    # Pivot the data: one column per Ticker, rows by Date
    pivot_df = df_selected.pivot(index='Date', columns='Ticker', values='Price')

    # Create interactive line plot
    fig = go.Figure()

    for ticker in pivot_df.columns:
        fig.add_trace(go.Scatter(
            x=pivot_df.index,
            y=pivot_df[ticker],
            mode='lines',
            name=ticker,
            opacity=0.7
        ))

    fig.update_layout(
        title='Stock Price Trends (2015 to Present) – Selected Sectors',
        xaxis_title='Date',
        yaxis_title='Closing Price (INR)',
        template='plotly_white',
        height=600
    )

    fig.show()
else:
    print("⚠️ No data found for the selected sectors in df_all")


### Stock Price Trends (2015 to Present) – Selected Sectors  

The above plot shows the stock price movement of selected Indian companies across sectors such as Banking, Pharma, FMCG, Auto, and Infrastructure.  


#### Interpretation of Results:
- **Strong Outperformers**:  
  - **BAJFINANCE.NS** and **BAJAJFINSV.NS** show the steepest upward trajectory, reflecting strong growth in the NBFC sector.  
  - **ADANIPORTS.NS** also demonstrates significant long-term appreciation.  

- **Banking Sector**:  
  - Large-cap banks such as **AXISBANK.NS** and **BANKBARODA.NS** have shown steady but less aggressive growth compared to NBFCs.  

- **Stable Performers**:  
  - Stocks like **ABB.NS** and **APOLLOHOSP.NS** display moderate upward movement, showing steady sectoral performance.  

- **Overall Trend**:  
  - Most companies show consistent appreciation since 2015, with sharper rallies visible post-2020, especially in finance-related stocks.  
  - This highlights sectoral divergence — financial services have outperformed traditional industrials and pharma in the observed period.  




In [ ]:
import plotly.graph_objects as go
import pandas as pd

# Extract selected sector names
sector_list = strong_sectors.index.tolist()

# Filter dataset
df_selected = df_all[df_all['Sector'].isin(sector_list)].copy()
df_selected['Date'] = pd.to_datetime(df_selected['Date'])

# Sector-wise plots
for sector in sector_list:
    df_sector = df_selected[df_selected['Sector'] == sector]
    pivot_sector = df_sector.pivot(index='Date', columns='Ticker', values='Price')
    
    if pivot_sector.empty:
        continue
    
    fig = go.Figure()
    for ticker in pivot_sector.columns:
        fig.add_trace(go.Scatter(x=pivot_sector.index, y=pivot_sector[ticker],
                                 mode='lines', name=ticker, opacity=0.8))
    
    fig.update_layout(
        title=f'Stock Price Trends – {sector}',
        xaxis_title='Date',
        yaxis_title='Closing Price (INR)',
        template='plotly_white',
        height=500
    )
    fig.show()




In [ ]:
# ---------------- Performance Metrics ----------------
# Calculate total return for each company
performance = (
    df_selected.groupby(["Sector", "Ticker"])["Price"]
    .agg(["first", "last"])
    .assign(ReturnPct=lambda x: (x["last"] - x["first"]) / x["first"] * 100)
    .reset_index()
)

# Sort companies within each sector by return
performance_sorted = performance.sort_values(["Sector", "ReturnPct"], ascending=[True, False])

print(performance_sorted)

### Performance Metrics

This cell calculates and ranks the performance of companies within each sector based on their price changes.

1. **Calculate Total Return for Each Company**
   - The data is grouped by `Sector` and `Ticker`.
   - For each company, the first and last prices are extracted.
   - The percentage return is computed as:  
     **ReturnPct = ((last price - first price) / first price) × 100**

2. **Sort Companies by Return**
   - Companies are sorted within each sector in descending order of their returns, so the best-performing companies appear first.

3. **Output**
   - The resulting `performance_sorted` DataFrame shows `Sector`, `Ticker`, `first price`, `last price`, and `ReturnPct` for each company in an organized manner.


In [ ]:

# ✅ Assuming df_all already loaded with columns: Date, Sector, Ticker, Price
df_all['Date'] = pd.to_datetime(df_all['Date'])

# ---- Step 1. Calculate daily returns ----
df_all['Return'] = df_all.groupby('Ticker')['Price'].pct_change()

# ---- Step 2. Aggregate metrics ----
metrics = []
for ticker, group in df_all.groupby('Ticker'):
    sector = group['Sector'].iloc[0]
    
    # Drop NA returns
    returns = group['Return'].dropna()
    
    if len(returns) == 0:
        continue
    
    # CAGR (approx from first & last price)
    start_price = group['Price'].iloc[0]
    end_price = group['Price'].iloc[-1]
    n_years = (group['Date'].iloc[-1] - group['Date'].iloc[0]).days / 365
    if n_years <= 0:
        continue
    cagr = (end_price / start_price) ** (1 / n_years) - 1
    
    # Volatility (annualized)
    volatility = returns.std() * np.sqrt(252)
    
    # Sharpe Ratio (risk-free assumed 0)
    sharpe = (returns.mean() * 252) / (returns.std() * np.sqrt(252))
    
    metrics.append([ticker, sector, cagr * 100, volatility * 100, sharpe])

# ---- Step 3. Build DataFrame ----
metrics_df = pd.DataFrame(metrics, columns=['Ticker', 'Sector', 'CAGR %', 'Volatility %', 'Sharpe Ratio'])

print("✅ metrics_df created")
print(metrics_df.head())


### Stock Performance Metrics Calculation

In this cell, we calculated key performance metrics for each stock in our dataset to analyze their historical performance and risk profile.  

1. **Daily Returns**:  
   - For each stock (`Ticker`), we computed the daily percentage change in price.  
   - This gives a sense of how volatile the stock is day-to-day.

2. **CAGR (Compound Annual Growth Rate)**:  
   - Using the first and last price along with the time period, we estimated the annual growth rate.  
   - This shows the average yearly return, smoothing out daily fluctuations.

3. **Volatility**:  
   - Calculated as the annualized standard deviation of daily returns.  
   - Higher volatility means the stock price swings more, indicating higher risk.

4. **Sharpe Ratio**:  
   - Measures risk-adjusted return, assuming a risk-free rate of zero.  
   - Higher Sharpe ratios indicate better return per unit of risk.

5. **Result**:  
   - All these metrics are compiled into the `metrics_df` DataFrame with columns: `Ticker`, `Sector`, `CAGR %`, `Volatility %`, `Sharpe Ratio`.  
   - The output shows the performance and risk characteristics for each stock, helping us identify well-performing and stable investments.


In [ ]:
final_companies = pick_companies_from_sectors(metrics_df, strong_sectors)

print(type(final_companies))  # should show <class 'pandas.core.frame.DataFrame'>
print(final_companies)


### Selecting Top Companies from Strong Sectors

In this cell, I filtered and selected companies from previously identified **strong sectors** using the `pick_companies_from_sectors` function.

1. **Purpose**:  
   - Focus only on companies within sectors that are performing well overall.  
   - Narrow down the list to high-potential stocks based on both **returns** and **risk-adjusted performance**.

2. **Output**:  
   - `final_companies` is a DataFrame containing the top companies from the strong sectors.  
   - Columns include:
     - `Ticker` – Stock symbol
     - `Sector` – Sector name
     - `CAGR %` – Annualized growth rate
     - `Volatility %` – Annualized price fluctuation
     - `Sharpe Ratio` – Risk-adjusted return

3. **Interpretation of Results**:  
   - Example: `IRFC.NS` in **Financial Services** has the highest CAGR of ~47.4% and a Sharpe Ratio of 1.15, indicating strong growth with reasonable risk.  
   - `VBL.NS` and `BAJFINANCE.NS` are also top performers in **FMCG** and **Financial Services** respectively.  
   - Companies like `TORNTPHARM.NS` and `DIVISLAB.NS` in **Healthcare** show moderate growth (~20%) but lower volatility, making them relatively stable options.  
   - The Sharpe Ratio column helps identify companies that offer the best return for the level of risk taken—the higher, the better.  

Overall, this selection highlights **high-growth and well-balanced companies** across strong sectors, useful for portfolio construction or focused analysis.


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12,6))
sns.barplot(
    data=final_companies,
    x="Ticker",
    y="Sharpe Ratio",
    hue="Sector",
    dodge=False,
    palette="Set2"
)
plt.title("Selected Companies by Sharpe Ratio")
plt.ylabel("Sharpe Ratio")
plt.xlabel("Company Ticker")
plt.xticks(rotation=45)
plt.legend(title="Sector", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


### Visualizing Selected Companies by Sharpe Ratio

This cell created a bar chart to compare the **risk-adjusted performance** of the selected companies (`final_companies`) across strong sectors.

- **X-axis**: `Ticker` – the stock symbol of each company.  
- **Y-axis**: `Sharpe Ratio` – higher values indicate better return relative to risk.  
- **Hue/Color**: `Sector` – different sectors are color-coded for easy comparison.  
- **Purpose**:  
  - Quickly identify which companies provide the best return per unit of risk.  
  - Compare risk-adjusted performance across sectors.   

The resulting chart helps visually spot top-performing companies and sector trends in terms of **risk-adjusted returns**.


In [ ]:
plt.figure(figsize=(10,6))
sns.scatterplot(
    data=final_companies,
    x="Volatility %",
    y="CAGR %",
    size="Sharpe Ratio",
    hue="Sector",
    palette="Set2",
    sizes=(100, 500),
    alpha=0.8
)
for i in range(final_companies.shape[0]):
    plt.text(final_companies["Volatility %"].iloc[i]+0.2, 
             final_companies["CAGR %"].iloc[i],
             final_companies["Ticker"].iloc[i],
             fontsize=9)
plt.title("Risk vs Return of Selected Companies")
plt.xlabel("Volatility (%)")
plt.ylabel("CAGR (%)")
plt.legend(title="Sector", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


### Risk vs Return of Selected Companies

This cell creates a scatter plot to visualize the relationship between **risk** and **return** for the selected companies (`final_companies`).

- **X-axis**: `Volatility %` – annualized risk of the stock.  
- **Y-axis**: `CAGR %` – annualized return of the stock.  
- **Size of Points**: `Sharpe Ratio` – larger points indicate higher risk-adjusted performance.  
- **Color/Hue**: `Sector` – different sectors are color-coded for easy comparison.  

**Additional Features**:  
- Each point is labeled with the company `Ticker` for easy identification.  
- `alpha=0.8` makes points slightly transparent for better visibility.  
- Legend is positioned outside the plot for clarity.  

**Purpose & Interpretation**:  
- Helps visualize the trade-off between **risk (volatility)** and **return (CAGR)**.  
- Companies in the **top-left** area (low volatility, high return) are attractive.  
- Point size highlights companies with higher **risk-adjusted returns (Sharpe Ratio)**.  
- Sector colors allow comparison of performance trends across industries.  

This plot provides a quick overview of which companies combine **strong growth** with **manageable risk**.


In [ ]:
final_companies["Sector"].value_counts().plot.pie(
    autopct='%1.1f%%',
    figsize=(6,6),
    colors=sns.color_palette("Set2"),
    startangle=140
)
plt.title("Sector Allocation of Selected Companies")
plt.ylabel("")
plt.show()


In [ ]:
sector_stats = final_companies.groupby("Sector")[["CAGR %", "Volatility %", "Sharpe Ratio"]].mean()
print(sector_stats)


In [ ]:
num_companies = len(final_companies)
final_companies["Weight"] = 1 / num_companies

# Portfolio expected return (CAGR weighted)
portfolio_cagr = (final_companies["CAGR %"] * final_companies["Weight"]).sum()

# Portfolio expected volatility (approx, assuming uncorrelated)
portfolio_vol = ((final_companies["Volatility %"]**2 * final_companies["Weight"]**2).sum())**0.5

# Portfolio Sharpe Ratio (simplified)
portfolio_sharpe = portfolio_cagr / portfolio_vol

print(f"Equal-Weighted Portfolio CAGR: {portfolio_cagr:.2f}%")
print(f"Equal-Weighted Portfolio Volatility: {portfolio_vol:.2f}%")
print(f"Equal-Weighted Portfolio Sharpe Ratio: {portfolio_sharpe:.2f}")


### Equal-Weighted Portfolio Performance

I constructed an **equal-weighted portfolio** using the selected companies, assigning each company an equal weight of **1 divided by the total number of companies**.

- **Portfolio Expected Return (CAGR):** 29.81%  
  The weighted average of individual company CAGR values indicates strong expected growth for the portfolio.

- **Portfolio Expected Volatility:** 10.84%  
  Assuming uncorrelated assets, the portfolio exhibits relatively low risk, demonstrating diversification benefits.

- **Portfolio Sharpe Ratio:** 2.75  
  This simplified Sharpe Ratio highlights **excellent risk-adjusted returns**, showing that the portfolio efficiently balances return and risk.

**Conclusion:** The equal-weighted portfolio of my selected companies is both high-returning and low-risk, further validating my company selection strategy.



In [ ]:
total_sharpe = final_companies["Sharpe Ratio"].sum()
final_companies["Weight_Sharpe"] = final_companies["Sharpe Ratio"] / total_sharpe

portfolio_cagr_sharpe = (final_companies["CAGR %"] * final_companies["Weight_Sharpe"]).sum()
portfolio_vol_sharpe = ((final_companies["Volatility %"]**2 * final_companies["Weight_Sharpe"]**2).sum())**0.5
portfolio_sharpe_weighted = portfolio_cagr_sharpe / portfolio_vol_sharpe

print(f"Sharpe-Weighted Portfolio CAGR: {portfolio_cagr_sharpe:.2f}%")
print(f"Sharpe-Weighted Portfolio Volatility: {portfolio_vol_sharpe:.2f}%")
print(f"Sharpe-Weighted Portfolio Sharpe Ratio: {portfolio_sharpe_weighted:.2f}")


### Sharpe-Weighted Portfolio Performance

I constructed a **Sharpe-weighted portfolio** by assigning each company a weight proportional to its **Sharpe Ratio**. This approach gives higher allocation to companies with better risk-adjusted returns.

- **Portfolio Expected Return (CAGR):** 31.05%  
  Weighting by Sharpe Ratio increases the contribution of high-performing companies, resulting in higher expected returns than the equal-weighted portfolio.

- **Portfolio Expected Volatility:** 11.17%  
  While slightly higher than the equal-weighted portfolio, the risk remains moderate due to diversification.

- **Portfolio Sharpe Ratio:** 2.78  
  The Sharpe-weighted portfolio achieves **superior risk-adjusted performance**, confirming that emphasizing companies with higher Sharpe Ratios improves portfolio efficiency.

**Conclusion:** By allocating weights based on Sharpe Ratios, my portfolio attains higher returns with marginally increased risk, effectively optimizing for risk-adjusted performance and further validating my company selection strategy.


In [ ]:
forecast_horizon = 365
forecast_dict = {}

for ticker in pivot_df.columns:
    series = pivot_df[ticker].dropna()

    try:
        model = ExponentialSmoothing(series, trend="add", seasonal=None)
        fit = model.fit()

        forecast_index = pd.date_range(
            start=pivot_df.index[-1] + pd.Timedelta(days=1),
            periods=forecast_horizon,
            freq="B"
        )
        forecast = pd.Series(fit.forecast(forecast_horizon).values, index=forecast_index)

        forecast_dict[ticker] = forecast

    except Exception as e:
        print(f"⚠️ Forecasting failed for {ticker}: {e}")

# --- Plot prices instead of returns ---
fig = go.Figure()

for ticker in pivot_df.columns:
    # Actual historical prices
    fig.add_trace(go.Scatter(
        x=pivot_df.index,
        y=pivot_df[ticker],
        mode='lines',
        name=f"{ticker} (Actual)"
    ))

    # Forecasted prices
    if ticker in forecast_dict:
        forecast_series = forecast_dict[ticker]
        fig.add_trace(go.Scatter(
            x=forecast_series.index,
            y=forecast_series.values,
            mode='lines',
            name=f"{ticker} (Forecast)",
            line=dict(dash='dash')
        ))

fig.update_layout(
    title="Stock Price Forecast (Next 90 Business Days)",
    xaxis_title="Date",
    yaxis_title="Price (INR)",
    template="plotly_white",
    height=650
)

fig.show()


### Stock Price Forecast (Next 365 Business Days)

I forecasted the prices of selected companies using **Exponential Smoothing** with an additive trend for **365 business days** beyond the latest historical data.

- **Actual prices:** solid lines  
- **Forecasted prices:** dashed lines  

**Interpretation:**  
- Upward forecasts indicate potential future growth.  
- Flat or declining forecasts suggest caution.  

**Purpose:**  
This forecast provides a **forward-looking view** to guide portfolio planning and investment decisions.


In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# === 1. Portfolio Data ===
df_all['Date'] = pd.to_datetime(df_all['Date'])

portfolio_tickers = final_companies['Ticker'].tolist()
df_portfolio = df_all[df_all['Ticker'].isin(portfolio_tickers)].copy()

# Filter last 5 years
cutoff = pd.Timestamp.today() - pd.DateOffset(years=5)
df_portfolio = df_portfolio[df_portfolio['Date'] >= cutoff]

# Pivot into price table
price_df = df_portfolio.pivot(index='Date', columns='Ticker', values='Price').dropna()

# Portfolio value (₹5000 invested equally)
initial_investment = 5000
weights = np.repeat(1/len(price_df.columns), len(price_df.columns))
normed = price_df / price_df.iloc[0]
portfolio_value = (normed * weights).sum(axis=1) * initial_investment

# === 2. Historical CAGR ===
years = (portfolio_value.index[-1] - portfolio_value.index[0]).days / 365
cagr = (portfolio_value.iloc[-1] / portfolio_value.iloc[0])**(1/years) - 1
print(f"Historical Portfolio CAGR: {cagr:.2%}")

# === 3. Projection Scenarios ===
bear_cagr = max(0.06, cagr/2)   # Conservative
base_cagr = cagr                # As-is
bull_cagr = cagr + 0.07         # Optimistic

scenarios = {
    "Bear Case": bear_cagr,
    "Base Case": base_cagr,
    "Bull Case": bull_cagr
}

# Projection horizon
future_years = [1, 3, 5]
today = portfolio_value.index[-1]

proj_df = pd.DataFrame(index=[today + pd.DateOffset(years=y) for y in future_years])

for name, rate in scenarios.items():
    proj_df[name] = [portfolio_value.iloc[-1] * ((1 + rate) ** y) for y in future_years]

# === 4. Plot History + Projections ===
fig = go.Figure()

# Historical portfolio
fig.add_trace(go.Scatter(
    x=portfolio_value.index,
    y=portfolio_value.values,
    mode='lines',
    name="Portfolio (Historical)",
    line=dict(color="black", width=2)
))

# Projection lines
for case in scenarios.keys():
    fig.add_trace(go.Scatter(
        x=[today] + proj_df.index.tolist(),
        y=[portfolio_value.iloc[-1]] + proj_df[case].tolist(),
        mode="lines+markers+text",
        text=[None] + [f"₹{v:,.0f}" for v in proj_df[case]],
        textposition="top center",
        name=case
    ))

fig.update_layout(
    title="Portfolio – Historical & Projected (₹5000 Invested)",
    xaxis_title="Date",
    yaxis_title="Portfolio Value (₹)",
    template="plotly_white",
    height=650
)

fig.show()


# 📊 Portfolio Backtest & Projections (₹5000 Invested)

### 1. Historical Analysis
- Data considered: **Last 5 years (from 2021-01-29 to 2025-09-10)**  
- Portfolio created from selected companies across strong sectors.  
- Equal weights applied to each stock.  
- **Historical CAGR:** 12.57%  

### 2. Projection Scenarios
Based on historical performance, we model 3 possible outcomes:

- 🐻 **Bear Case:** Conservative CAGR (minimum 6% or half of historical CAGR)  
- 📈 **Base Case:** CAGR equal to historical performance  
- 🚀 **Bull Case:** Optimistic CAGR = historical + 7%  

### 3. Results (Portfolio Value from ₹5000 invested)
| Horizon | Bear Case | Base Case | Bull Case |
|---------|-----------|-----------|-----------|
| 1 Year  | ₹27,717   | ₹32,277   | ₹33,898   |
| 3 Years | ₹39,707   | ₹62,705   | ₹72,635   |
| 5 Years | ₹56,884   | ₹121,819  | ₹155,639  |

### 4. Visualization
- **Black Line:** Historical portfolio growth from 2020 to present.  
- **Colored Lines:** Future projections (Bear, Base, Bull).  
- Each projection shows the expected portfolio value at **1, 3, and 5 years**.  

---

💡 This approach demonstrates how ₹5000 invested in the chosen sectors could evolve under different market scenarios.  
It provides investors with **clear downside protection (Bear)**, **realistic expectation (Base)**, and **optimistic growth potential (Bull)**.


### ✅ Key Takeaways

- The selected companies across strong sectors provide **high historical CAGR** with **moderate risk**, validated through portfolio backtesting.  
- Both **equal-weighted** and **Sharpe-weighted** portfolios show strong **risk-adjusted returns**, with the Sharpe-weighted portfolio slightly optimizing returns.  
- Scenario analysis (Bear, Base, Bull) provides **clear insight into potential portfolio outcomes**, helping in planning and risk management.  
- The **stock price forecasts** and historical trends offer additional guidance for **future investment decisions**.  

**Conclusion:**  
This analysis demonstrates that careful **company selection and portfolio construction** can generate **consistent returns** while managing risk. Investors can use these insights to **make informed, data-driven decisions** for both short-term and long-term investment planning.


In [1]:
!pip install nbstripout

In [2]:
nbstripout .ipynb


SyntaxError: invalid syntax (932712763.py, line 1)